In [1]:

from sklearn.datasets import fetch_openml
import numpy as np
from types import SimpleNamespace
from typing import Tuple, List
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
    
adult = fetch_openml(data_id = 531, data_home='./data_cache')

data = adult.data

le = LabelEncoder()
label = pd.Series(le.fit_transform(adult.target))


category_cols = ["CHAS"]
continuous_cols = [x for x in data.columns if x not in category_cols]

for col in category_cols:
    data[col] = le.fit_transform(data[col])


    
    
temp = None
for col in category_cols:
    oh_values = OneHotEncoder().fit_transform(data[col].values.reshape((-1, 1))).toarray()
    new_cols = [col + "-" + str(i) for i in range(len(data[col].unique()))]
    oh_values = pd.DataFrame(oh_values, columns = new_cols, dtype=np.int8, index=data.index)
    if temp is None:
        temp = oh_values
    else:
        temp = temp.merge(oh_values, left_index=True, right_index=True)

data = data.merge(temp, left_index=True, right_index=True)
data.drop(category_cols, inplace=True, axis=1)

category_cols = temp.columns

scaler = MinMaxScaler()
data[continuous_cols] = scaler.fit_transform(data[continuous_cols])

/opt/conda/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [2]:
import torch.nn as nn
model_hparams = {
    "encoder_dim" : data.shape[1],
    "predictor_hidden_dim" : 256,
    "predictor_output_dim" : 1,
    'alpha1' : 0.5,
    'alpha2' : 0.5,
    'beta' : 0.5,
    'K' : 10
}
data_hparams = {
    "K" : 10,
    "p_m" : 0.2
}
optim_hparams = {
    "lr" : 0.005
}
scheduler_hparams = {
    'scheduler_gamma' : 0.3,
    'scheduler_step_size' : 30
}
num_categoricals = len(continuous_cols)
num_continuous = len(continuous_cols)
consistency_loss = nn.MSELoss
loss_fn = nn.MSELoss
metric =  "mean_squared_error"
random_seed = 0

In [3]:
from misc.scorer import BaseScorer


class MSEScorer(BaseScorer):
    def __init__(self, metric: str) -> None:
        super().__init__(metric)
    
    def __call__(self, y, y_hat) -> float:
        return self.metric(y, y_hat)

In [4]:
from pl_vime import PLVIME
pl_vime = PLVIME(model_hparams, optim_hparams, scheduler_hparams, 
       num_categoricals, num_continuous, -1,  consistency_loss, loss_fn,
       MSEScorer("mean_squared_error"), random_seed)

Global seed set to 0


In [5]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(data, label, train_size = 0.7, random_state=random_seed)

X_train, X_unlabeled, y_train, _ = train_test_split(X_train, y_train, train_size = 0.3, random_state=random_seed)

In [6]:
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from data_utils import *

n_gpus = 1
n_jobs = 32
max_epochs = 10
batch_size = 128

pretraining_patience = 10
early_stopping_patience = 30

def fit_model(
            model,
    ):
    
    train_ds = VIMESelfDataset(X_train.append(X_unlabeled), data_hparams, continuous_cols, category_cols)
    test_ds = VIMESelfDataset(X_valid, data_hparams, continuous_cols, category_cols)
    
    pl_datamodule = PLDataModule(train_ds, test_ds, batch_size=batch_size, is_regression=True)

    model.do_pretraining()

    callbacks = [
        EarlyStopping(
            monitor= 'val_loss', 
            mode = 'min',
            patience = pretraining_patience,
            verbose = False
        )
    ]
    pretraining_path = f'temporary_ckpt_data/pretraining'
    checkpoint_callback = ModelCheckpoint(
        monitor='val_loss',
        dirpath=pretraining_path,
        filename='pretraining-{epoch:02d}-{val_f1:.4f}',
        save_top_k=1,
        mode = 'min'
    )

    callbacks.append(checkpoint_callback)

    trainer = Trainer(
                    devices = n_gpus,
                    accelerator="cuda" if n_gpus >= 1 else 'cpu',
                    # replace_sampler_ddp=False,
                    max_epochs = max_epochs,
                    num_sanity_val_steps = 2,
                    callbacks = callbacks,
    )

    trainer.fit(model, pl_datamodule)
    
    pretraining_path = checkpoint_callback.best_model_path

    model = model.load_from_checkpoint(pretraining_path)

    model.do_finetuning()
    
        
    train_ds = VIMERegressionDataset(X_train, y_train.values, data_hparams, X_unlabeled, continuous_cols, category_cols)
    test_ds = VIMERegressionDataset(X_valid, y_valid.values, data_hparams, None, continuous_cols, category_cols)

    pl_datamodule = PLDataModule(train_ds, test_ds, batch_size = batch_size, is_regression=True, n_jobs=1)
        
    callbacks = [
        EarlyStopping(
            monitor= 'val_' + metric, 
            mode = 'max',
            patience = early_stopping_patience,
            verbose = False
        )
    ]

    checkpoint_path = None

    checkpoint_path = f'temporary_ckpt_data/'
    checkpoint_callback = ModelCheckpoint(
        monitor='val_' + metric,
        dirpath=checkpoint_path,
        filename='{epoch:02d}-{val_f1:.4f}',
        save_top_k=1,
        mode = 'max'
    )

    callbacks.append(checkpoint_callback)

    trainer = Trainer(
                    devices = n_gpus,
                    accelerator = "cuda" if n_gpus >= 1 else 'cpu',
                    # replace_sampler_ddp=False,
                    max_epochs = max_epochs,
                    num_sanity_val_steps = 2,
                    callbacks = callbacks,
    )

    trainer.fit(model, pl_datamodule)

    model = model.load_from_checkpoint(checkpoint_callback.best_model_path)
    
    return model

In [7]:
pl_vime = fit_model(pl_vime)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 4090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
/opt/conda/envs/py37/lib/python3.7/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:613: UserWarning: Checkpoint directory /workspace/vime/temporary_ckpt_data/pretraining exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name                      | Type             | Params
---------------------------------------------------------------
0 | model                     |

Sanity Checking: 0it [00:00, ?it/s]

/opt/conda/envs/py37/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py:1613: PossibleUserWarning: The number of training batches (3) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  category=PossibleUserWarning,


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

/opt/conda/envs/py37/lib/python3.7/site-packages/torch/nn/modules/loss.py:530: UserWarning: Using a target size (torch.Size([10, 1])) that is different to the input size (torch.Size([10])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Validation: 0it [00:00, ?it/s]

/opt/conda/envs/py37/lib/python3.7/site-packages/torch/nn/modules/loss.py:530: UserWarning: Using a target size (torch.Size([10, 1])) that is different to the input size (torch.Size([10])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.
Global seed set to 0


In [8]:
trainer = Trainer(
                    devices = n_gpus,
                    accelerator = "cuda" if n_gpus >= 1 else 'cpu',
                    # replace_sampler_ddp=False,
                    max_epochs = max_epochs,
                    num_sanity_val_steps = 2,
                    callbacks = None,
    )
test_ds = VIMERegressionDataset(X_valid, y_valid.values, data_hparams, None, continuous_cols, category_cols)
test_dl = DataLoader(test_ds, batch_size, shuffle=False, sampler = SequentialSampler(test_ds), num_workers=n_jobs)

pl_vime.do_finetuning()

preds = trainer.predict(pl_vime, test_dl)

preds = np.concatenate([out.cpu().numpy() for out in preds])


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 4090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Predicting: 0it [00:00, ?it/s]

In [9]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_valid, preds.ravel())

13981.019278119371

In [10]:
with torch.no_grad():
    pl_vime.do_finetuning()
    preds = pl_vime(torch.from_numpy(X_valid.values).to(torch.float32))

In [11]:
with torch.no_grad():
    out = pl_vime.model.self_sl.h(torch.from_numpy(X_valid.values).to(torch.float32))
    out = pl_vime.model.semi_sl(out)